# LongT5 with color tranining, weird output, testing against model in the memory

2024 11 12 16:20

Same as 20241111-color-test-longt5, except in the dn I tested the inference with a model in the memory vs the one loaded. The one in the memory did great, but when it was loaded from the disk it returned gibberish.


In [1]:
import shutil
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U git+https://github.com/huggingface/accelerate.git
!pip install -U git+https://github.com/huggingface/datasets.git
!pip install -U git+https://github.com/huggingface/evaluate.git

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-akv5zoz8
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-akv5zoz8
  Resolved https://github.com/huggingface/transformers.git to commit 33eef992503689ba1af98090e26d3e98865b2a9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 11.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 113.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10053251 sha256=08bfbd27d2eafda6048d39d

In [2]:
!apt install zip -y
!rm -rf data-rb-color
!mkdir -p data-rb-color
!wget "https://www.dropbox.com/scl/fi/vd0ypt9mo9oh0p9tf90h3/dataset-rb-color-fixed.zip?rlkey=bieseudpp5pzko5j4u1n67phq&dl=1" -O model.zip
!unzip model.zip -d data-rb-color

!mkdir -p data-rb-validate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  unzip
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 27 not upgraded.
Need to get 350 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2 [175 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 zip amd64 3.0-12build2 [176 kB]
Fetched 350 kB in 1s (480 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 34250 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 11%] [######................................

In [3]:
from datasets import load_from_disk

dataset = load_from_disk('data-rb-color')
dataset = dataset.train_test_split(test_size=0.00004) 


In [4]:
print(dataset['train'][0])

{'svg': '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="393" height="852" viewBox="0 0 393 852"><g id="html1"><g data-tag="head" id="head1" data-z-index="auto" data-stacking-context="true" aria-owns="script1"><g data-tag="script" id="script1" data-z-index="auto" data-stacking-context="true"/></g><g data-tag="body" id="body1" data-z-index="auto" data-stacking-context="true" role="document" aria-owns="style1"><g data-stacking-layer="rootBackgroundAndBorders"><rect width="377" height="836" x="8" y="8" fill="rgb(34, 177, 219)"/></g><g data-tag="style" id="style1" data-z-index="auto" data-stacking-context="true"/></g></g></svg>', 'html': '<body></body>\n\n<style>\n\n        body {\n            background-color: #22b1db;\n        }\n    \n</style>'}


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
new_words = ['<', '>', '{', '}', '<body>', '<style>', '</style>', '</body>']
tokenizer.add_tokens(new_words)

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

8

In [6]:
tokenized_output = tokenizer.encode(dataset['train'][0]['svg'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized SVG contains unknown tokens.")
else:
    print("No unknown tokens found in the SVG.")

tokenized_output = tokenizer.encode(dataset['train'][0]['html'], add_special_tokens=False)
unk_token_id = tokenizer.unk_token_id
if unk_token_id in tokenized_output:
    print("Alert: The tokenized HTML contains unknown tokens.")
else:
    print("No unknown tokens found in the HTML.")

No unknown tokens found in the SVG.
No unknown tokens found in the HTML.


In [13]:
MAX_SEQUENCE_LENGTH = 512

def get_token_lengths(examples):
    inputs = tokenizer(
        examples['svg'],
        truncation=False,  # Don't truncate yet
        padding=False,     # Don't pad yet
        return_length=True,
    )
    labels = tokenizer(
        examples['html'],
        truncation=False,
        padding=False,
        return_length=True,
    )

    inputs["input_length"] = inputs["length"]
    inputs["labels"] = labels["input_ids"]
    inputs["label_length"] = labels["length"]

    inputs.pop("length", None)

    return inputs

tokenized_data = dataset.map(get_token_lengths, batched=True)

def filter_function(example):
    return example['input_length'] <= MAX_SEQUENCE_LENGTH and example['label_length'] <= MAX_SEQUENCE_LENGTH

filtered_data = tokenized_data.filter(filter_function)

def tokenize_examples(examples):
    inputs = tokenizer(
        examples['svg'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )
    labels = tokenizer(
        examples['html'],
        max_length=MAX_SEQUENCE_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
    )

    labels = labels['input_ids']
    labels[labels == tokenizer.pad_token_id] = -100
    inputs["labels"] = labels
    

    return inputs

filtered_data = filtered_data.map(get_token_lengths, batched=True, remove_columns=dataset["train"].column_names)

print(filtered_data)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 99996
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'input_length', 'labels', 'label_length'],
        num_rows: 4
    })
})


In [14]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def create_compute_metrics(trainer):
    def compute_metrics(eval_predictions):
        predictions, labels = eval_predictions
        if isinstance(predictions, tuple):
            predictions = predictions[0]
            
        predictions = predictions[-4:]
        labels = labels[-4:]
            
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
        decoded_predictions, decoded_labels = postprocess_text(decoded_predictions, decoded_labels)
    
        similarity_scores = []
        perceptual_losses = []
        index = 1
        for prediction, label in zip(decoded_predictions, decoded_labels):
            metrics = calculate_metrics(prediction, label[0])
            similarity_scores.append(metrics['similarity'])
            perceptual_losses.append(metrics['perceptual_loss'])
            
            add_image_to_tensorboard(f'valid_{index}_expectation', trainer.state.global_step, metrics['expected_screenshot_path'])
            add_image_to_tensorboard(f'valid_{index}_prediction', trainer.state.global_step, metrics['predicted_screenshot_path'])
            
            index += 1
        
        result = {
            'similarity': float(np.mean(similarity_scores)),
            'perceptual_loss': float(np.mean(perceptual_losses)),
        }
    
        result = {k: round(v, 4) for k, v in result.items()}
        return result
    
    return compute_metrics

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="google/long-t5-tglobal-base")

In [16]:
from transformers import AutoModelForSeq2SeqLM, AutoConfig

config = AutoConfig.from_pretrained(
    "google/long-t5-tglobal-base",
    vocab_size=len(tokenizer),
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    decoder_start_token_id = tokenizer.pad_token_id,
)

# model = AutoModelForSeq2SeqLM.from_pretrained("google/long-t5-tglobal-base")
model = AutoModelForSeq2SeqLM.from_config(config)

# model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")
model.resize_token_embeddings(len(tokenizer))

Embedding(32107, 768)

In [17]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_model_config(model.config)
generation_config._from_model_config = False
generation_config.max_new_tokens = 512

generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "max_new_tokens": 512,
  "pad_token_id": 0
}

In [18]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_output_dir = f"model-{timestamp}"
log_dir = f'{model_output_dir}/runs'

%load_ext tensorboard
%tensorboard --logdir '{log_dir}' --port=6006

In [19]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=model_output_dir,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # gradient_accumulation_steps=16,
    # gradient_checkpointing=True,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    logging_strategy="steps",
    logging_steps=25,
    logging_dir=log_dir,
    predict_with_generate=True,
    # load_best_model_at_end=True,
    bf16=True,
    generation_config=generation_config,
    report_to="tensorboard"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=filtered_data["train"],
    eval_dataset=filtered_data["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
)

trainer.compute_metrics = create_compute_metrics(trainer)

trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Similarity,Perceptual Loss,Runtime,Samples Per Second,Steps Per Second
1000,6.183000,4.535502,0.749800,0.557200,20.704400,0.193000,0.048000
2000,5.420800,4.018407,0.882400,0.507000,18.962000,0.211000,0.053000
3000,5.056700,4.052268,0.870700,0.439100,18.904100,0.212000,0.053000
4000,4.287800,3.708380,0.818000,0.512400,19.207200,0.208000,0.052000
5000,3.780500,3.291784,0.897800,0.436500,18.740200,0.213000,0.053000
6000,3.220900,2.702464,0.858700,0.373000,18.681100,0.214000,0.054000
7000,2.585800,3.349219,0.749800,0.557200,18.761100,0.213000,0.053000
8000,2.421300,2.737944,0.855800,0.386700,19.317900,0.207000,0.052000
9000,2.243300,2.692992,0.811700,0.433600,18.883700,0.212000,0.053000
10000,1.943900,2.299574,0.830200,0.281500,18.831400,0.212000,0.053000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


<body></body><style> body{ background-color: #76a266;}</style>
62
<body></body><style> body{ background-color: #ddddddddd;}</style>
65
Serving on port 8893


127.0.0.1 - - [12/Nov/2024 03:47:01] "GET /predicted.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 03:47:02] "GET /expected.html HTTP/1.1" 200 -


...

127.0.0.1 - - [12/Nov/2024 06:28:54] "GET /predicted.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 06:28:55] "GET /expected.html HTTP/1.1" 200 -


Output:  {"similarity": 1.0, "perceptual_loss": 0.0}
<body></body><style> body{ background-color: #990fe4;}</style>
62
<body></body><style> body{ background-color: #990fe4;}</style>
62


127.0.0.1 - - [12/Nov/2024 06:28:59] "GET /predicted.html HTTP/1.1" 200 -
127.0.0.1 - - [12/Nov/2024 06:28:59] "GET /expected.html HTTP/1.1" 200 -


Output:  {"similarity": 1.0, "perceptual_loss": 0.0}


TrainOutput(global_step=62500, training_loss=0.9688850410623551, metrics={'train_runtime': 9937.6331, 'train_samples_per_second': 100.624, 'train_steps_per_second': 6.289, 'total_flos': 4.640055464429568e+17, 'train_loss': 0.9688850410623551, 'epoch': 10.0})

In [22]:
import os
from transformers import LongT5ForConditionalGeneration

text = dataset['test'][0]['svg']

inputs = tokenizer(text, return_tensors="pt").input_ids
inputs = inputs.to('cuda:0')

model.eval()

with torch.no_grad():
    model.to('cuda:0')
    model.generation_config = generation_config
    outputs = model.generate(inputs)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translation)

print(dataset['test'][0]['html'])

<body></body><style> body{ background-color: #76a266;}</style>
<body></body>

<style>

        body {
            background-color: #76a266;
        }
    
</style>


In [24]:
import os 

model = LongT5ForConditionalGeneration.from_pretrained(os.path.join(model_output_dir, "checkpoint-62500"))
model.to('cuda:0')
model.generation_config = generation_config

outputs = model.generate(inputs)

translation = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translation)

98989898989898989898989898989898989898989898989898989898989898989898989898;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;98;
